In [2]:
import sage.all

In [10]:
def better_bn(n):
    from itertools import product, permutations
    from sage.matrix.constructor import matrix

    signs = list(product([-1, 1], repeat=n))
    

    perms = list(permutations(range(n)))

    

    matrices = [matrix(n, n, [s[p[i]] if j == p[i] else 0 for i in range(n) for j in range(n)]) for s in signs for p in perms]

    return matrices

G = better_bn(3)
G
# matrix = MatrixGroup(G)



# table = matrix.character_table()
# table

 
#below code used for large B_n, doesn't print properly with built-in commands
    
# for i in range(table.nrows()):
#     for j in range(table.ncols()):
#         print(table[i,j], end='  ')  # Adjust spacing as needed
#     print()  # To move to the next line

#table.str()

[[-1  0  0]
 [ 0 -1  0]
 [ 0  0 -1],
 [-1  0  0]
 [ 0  0 -1]
 [ 0 -1  0],
 [ 0 -1  0]
 [-1  0  0]
 [ 0  0 -1],
 [ 0 -1  0]
 [ 0  0 -1]
 [-1  0  0],
 [ 0  0 -1]
 [-1  0  0]
 [ 0 -1  0],
 [ 0  0 -1]
 [ 0 -1  0]
 [-1  0  0],
 [-1  0  0]
 [ 0 -1  0]
 [ 0  0  1],
 [-1  0  0]
 [ 0  0  1]
 [ 0 -1  0],
 [ 0 -1  0]
 [-1  0  0]
 [ 0  0  1],
 [ 0 -1  0]
 [ 0  0  1]
 [-1  0  0],
 [ 0  0  1]
 [-1  0  0]
 [ 0 -1  0],
 [ 0  0  1]
 [ 0 -1  0]
 [-1  0  0],
 [-1  0  0]
 [ 0  1  0]
 [ 0  0 -1],
 [-1  0  0]
 [ 0  0 -1]
 [ 0  1  0],
 [ 0  1  0]
 [-1  0  0]
 [ 0  0 -1],
 [ 0  1  0]
 [ 0  0 -1]
 [-1  0  0],
 [ 0  0 -1]
 [-1  0  0]
 [ 0  1  0],
 [ 0  0 -1]
 [ 0  1  0]
 [-1  0  0],
 [-1  0  0]
 [ 0  1  0]
 [ 0  0  1],
 [-1  0  0]
 [ 0  0  1]
 [ 0  1  0],
 [ 0  1  0]
 [-1  0  0]
 [ 0  0  1],
 [ 0  1  0]
 [ 0  0  1]
 [-1  0  0],
 [ 0  0  1]
 [-1  0  0]
 [ 0  1  0],
 [ 0  0  1]
 [ 0  1  0]
 [-1  0  0],
 [ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1],
 [ 1  0  0]
 [ 0  0 -1]
 [ 0 -1  0],
 [ 0 -1  0]
 [ 1  0  0]
 [ 0  0 -1],
 

In [8]:
from sage.combinat.root_system.weyl_group import WeylGroup

n = 9
W = WeylGroup(["B", n])
char_table = W.character_table()
print(char_table)

CT2

       2   16   16   14   14   15   15   14   14   16    16  12 10 11 10  12
       3    4    2    2    3    2    2    3    2    2     4   1  1  .  1   1
       5    1    1    1    1    1    1    1    1    1     1   .  .  .  .   .
       7    1    1    1    .    .    .    .    1    1     1   .  .  .  .   .

           1a   2a   2b   2c   2d   2e   2f   2g   2h    2i  2j 4a 4b 4c  4d

X.1         1    1    1    1    1    1    1    1    1     1   1  1  1  1   1
X.2         1   -1    1   -1    1   -1    1   -1    1    -1   1 -1  1 -1   1
X.3         1   -1    1   -1    1   -1    1   -1    1    -1   1 -1  1 -1   1
X.4         1    1    1    1    1    1    1    1    1     1   1  1  1  1   1
X.5         8    8    8    8    8    8    8    8    8     8   .  .  .  .   .
X.6         8    8    8    8    8    8    8    8    8     8   .  .  .  .   .
X.7         8   -8    8   -8    8   -8    8   -8    8    -8   .  .  .  .   .
X.8         8   -8    8   -8    8   -8    8   -8    8    -8   .  .  .

The above is how you'd do this using Sage's built in stuff…

In [32]:

def tuple_rep(matrix):
    n = matrix.nrows()
    left = []
    right = []
    visited = set()

    for i in range(n):
        if i in visited:
            continue
        
        cycle = 0
        parity = 1
        row = i

        while row != i or cycle == 0:
            visited.add(row)

            for j, elt in enumerate(matrix.row(row)):
                if elt != 0:
                    cycle += 1
                    parity *= elt
                    row = j
                    break

        if parity == 1:
            left.append(cycle)
        else:
            right.append(cycle)
    
    return [tuple(sorted(left, reverse=True)), tuple(sorted(right, reverse=True))]

tuple_rep(G[1])

[(2,), (1,)]

In [87]:
# def find_congruent_classes(char_table, p_m):
#     # Get the number of rows and columns in the character table
#     num_rows, num_cols = char_table.dimensions()

#     # Initialize a dictionary to store the congruent classes
#     congruent_classes = {}

# #     # Initialize a set to store the indices of columns that have been added to the dictionary
# #     added_columns = set()

#     # Iterate over the columns of the character table
#     for i in range(num_cols):
# #         if i not in added_columns:
#             for j in range(i+1, num_cols):
#                 # Check if the two columns are congruent modulo p_m
#                 if all((ZZ(char_table[k,i]) - ZZ(char_table[k,j])) % p_m == 0 for k in range(num_rows)):
#                     # If they are, add them to the dictionary
#                     if i not in congruent_classes:
#                         congruent_classes[i] = [char_table.column(i)]
# #                         added_columns.add(i)
#                     congruent_classes[i].append(char_table.column(j))
#                 else:
#                     if i not in congruent_classes:
#                         congruent_classes[i] = [char_table.column(i)]
# #                     added_columns.add(j)



#     # Return the dictionary of congruent classes
#     return congruent_classes


In [88]:
def find_congruent_classes_v2(G, char_table, p_m):
    # Get the number of rows and columns in the character table
    num_rows, num_cols = char_table.dimensions()

    # Initialize a dictionary to store the congruent classes
    congruent_classes = {}
    
    conjugacy_classes = G.conjugacy_classes()
#     # Initialize a set to store the indices of columns that have been added to the dictionary
#     added_columns = set()

    # Iterate over the columns of the character table
    for i in range(num_cols):
#         if i not in added_columns:
            for j in range(i+1, num_cols):
                # Check if the two columns are congruent modulo p_m
                if all((ZZ(char_table[k,i]) - ZZ(char_table[k,j])) % p_m == 0 for k in range(num_rows)):
                    # If they are, add them to the dictionary
                    if str(conjugacy_classes[i]) not in congruent_classes:
                        congruent_classes[str(conjugacy_classes[i])] = [char_table.column(i)]
#                         added_columns.add(i)
                    congruent_classes[str(conjugacy_classes[i])].append(char_table.column(j))
                else:
                    if str(conjugacy_classes[i]) not in congruent_classes:
                        congruent_classes[str(conjugacy_classes[i])] = [char_table.column(i)]
#                     added_columns.add(j)



    # Return the dictionary of congruent classes
    return congruent_classes


In [89]:
p = 2
m = 2

result = find_congruent_classes_v2(matrix, table, 2)


added_columns = set()
toReturnDict = {}

for key in result:
    toReturnDict[key] = []
    for column in result[key]:
        if tuple(column) not in added_columns:
            toReturnDict[key].append(column)
            added_columns.add(tuple(column))

clean_dict = {}

for key in toReturnDict:
    if toReturnDict[key] != []:
        clean_dict[key] = toReturnDict[key]
        
print(clean_dict)


{'Conjugacy class of [1 0 0]\n[0 1 0]\n[0 0 1] in Matrix group over Integer Ring with 48 generators': [(1, 1, 1, 1, 2, 2, 3, 3, 3, 3), (1, 1, 1, 1, 2, 2, -1, -1, -1, -1), (1, -1, -1, 1, 0, 0, -1, -1, 1, 1), (1, 1, -1, -1, 0, 0, 1, -1, -1, 1), (1, -1, 1, -1, -2, 2, -1, 1, -1, 1), (1, 1, -1, -1, 0, 0, -1, 1, 1, -1), (1, -1, -1, 1, 0, 0, 1, 1, -1, -1), (1, -1, 1, -1, -2, 2, 3, -3, 3, -3)], 'Conjugacy class of [ 0 -1  0]\n[ 0  0 -1]\n[-1  0  0] in Matrix group over Integer Ring with 48 generators': [(1, -1, 1, -1, 1, -1, 0, 0, 0, 0), (1, 1, 1, 1, -1, -1, 0, 0, 0, 0)]}
